# Decision Analysis Project 2

- Mateusz Tabaszewski 151945
- Bartłomiej Pukacki 151942

This is our notebook dedicated to the second project from decision analysis classes, in this notebook we set out to implement our own versions of UTA and AHP methods used for multi-criteria decision analysis (MCDA). 

Sections present in this notebook:
* Imports - imports used for the notebook
* Utilities - general functions and classes which can be used outside the implemented methods
* Data - showcase of the constructed dataset
* UTA - implementation of UTA method
* AHP - implementation of AHP method
* Use - comparison of methods including PROMETHEE II results

## Imports

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Utilities

## Data


id | name                          | price | critic_score | user_score | length | genres   | num_of_achievements |
---|-------------------------------|-------|--------------|------------|--------|----------|---------------------|
0  | Dark Souls: Remastered        | 150   | 84           | 83         | 44     | 11       | 41                  |
1  | Dark Souls III                | 200   | 89           | 90         | 49     | 11       | 43                  |
2  | Terraria                      | 46    | 81           | 81         | 102    | 9        | 115                 |
3  | Baldur's Gate 3               | 250   | 96           | 89         | 107    | 8        | 54                  |
4  | Dave the Diver                | 92    | 90           | 83         | 32     | 7        | 43                  |
5  | Rust                          | 153   | 69           | 65         | 37     | 10       | 92                  |
6  | Hollow Knight                 | 68    | 90           | 91         | 42     | 10       | 63                  |
7  | Portal 2                      | 46    | 95           | 89         | 14     | 7        | 51                  |
8  | Vampire Survivors             | 20    | 86           | 83         | 25     | 9        | 204                 |
9  | Hades                         | 115   | 93           | 88         | 49     | 9        | 49                  |
10 | Subnautica                    | 139   | 87           | 86         | 43     | 5        | 17                  |
11 | Dishonored                    | 45    | 88           | 83         | 18     | 8        | 80                  |
12 | Ori and the Will of the Wisps | 108   | 90           | 89         | 16     | 10       | 37                  |
13 | Inside                        | 72    | 93           | 83         | 4      | 6        | 14                  |
14 | The Forest                    | 72    | 83           | 75         | 28     | 9        | 45                  |
15 | Skyrim                        | 90    | 96           | 86         | 114    | 11       | 75                  |
16 | Teardown                      | 120   | 80           | 81         | 22     | 6        | 27                  |
17 | Dying Light                   | 90    | 74           | 81         | 36     | 5        | 78                  |
18 | Enter the Gungeon             | 68    | 82           | 80         | 62     | 4        | 54                  |
19 | Payday 3                      | 169   | 66           | 31         | 10     | 5        | 22                  |
20 | Kao the Kangaroo              | 129   | 65           | 75         | 8      | 8        | 26                  |
21 | Assassin's Creed Unity        | 120   | 72           | 56         | 35     | 11       | 57                  |
22 | Trials Fusion                 | 80    | 79           | 71         | 23     | 2        | 51                  |
23 | The Sims 3                    | 28    | 83           | 78         | 78     | 3        | 65                  |
24 | Titan Souls                   | 68    | 74           | 61         | 4      | 8        | 27                  |

In [16]:
data = pd.read_csv("data.csv")
data

,id,name,price,critic_score,user_score,length,genres,num_of_achievements
0,0,Dark Souls: Remastered,150,84,83,44,11,41
1,1,Dark Souls III,200,89,90,49,11,43
2,2,Terraria,46,81,81,102,9,115
3,3,Baldur's Gate 3,250,96,89,107,8,54
4,4,Dave the Diver,92,90,83,32,7,43
5,5,Rust,153,69,65,37,10,92
6,6,Hollow Knight,68,90,91,42,10,63
7,7,Portal 2,46,95,89,14,7,51
8,8,Vampire Survivors,20,86,83,25,9,204
9,9,Hades,115,93,88,49,9,49


## UTA

## AHP

### Criteria hierarchy:

Goal: buying a video game that is described by quality and quantity measures

```tree
goal
├── quality
│   ├── user_score
│   ├── critic_score
│   └── genres
└── quantity
    ├── price
    ├── length
    └── num_of_achievements
```

### Pairwise comparisons of hierarchy elements

| user_score | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 |
|------------|---|---|---|---|---|---|---|---|---|---|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
| 0          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 1          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 2          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 3          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 4          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 5          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 6          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 7          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 8          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 9          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 10         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 11         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 12         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 13         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 14         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 15         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 16         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 17         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 18         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 19         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 20         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 21         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 22         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 23         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| 24         | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |

In [24]:
proportions = []
for i in range(len(data)):
    row = []
    for j in range(len(data)):
        if data.at[j, 'user_score'] != 0:
            row.append(data.at[i, 'user_score'] / data.at[j, 'user_score'])
        else:
            row.append(None)
    proportions.append(row)

df_proportions = pd.DataFrame(proportions, columns=data['id'], index=data['id'])
df_proportions

id,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.922222,1.024691,0.932584,1.000000,1.276923,0.912088,0.932584,1.000000,0.943182,...,0.965116,1.024691,1.024691,1.0375,2.677419,1.106667,1.482143,1.169014,1.064103,1.360656
1,1.084337,1.000000,1.111111,1.011236,1.084337,1.384615,0.989011,1.011236,1.084337,1.022727,...,1.046512,1.111111,1.111111,1.1250,2.903226,1.200000,1.607143,1.267606,1.153846,1.475410
2,0.975904,0.900000,1.000000,0.910112,0.975904,1.246154,0.890110,0.910112,0.975904,0.920455,...,0.941860,1.000000,1.000000,1.0125,2.612903,1.080000,1.446429,1.140845,1.038462,1.327869
3,1.072289,0.988889,1.098765,1.000000,1.072289,1.369231,0.978022,1.000000,1.072289,1.011364,...,1.034884,1.098765,1.098765,1.1125,2.870968,1.186667,1.589286,1.253521,1.141026,1.459016
4,1.000000,0.922222,1.024691,0.932584,1.000000,1.276923,0.912088,0.932584,1.000000,0.943182,...,0.965116,1.024691,1.024691,1.0375,2.677419,1.106667,1.482143,1.169014,1.064103,1.360656
5,0.783133,0.722222,0.802469,0.730337,0.783133,1.000000,0.714286,0.730337,0.783133,0.738636,...,0.755814,0.802469,0.802469,0.8125,2.096774,0.866667,1.160714,0.915493,0.833333,1.065574
6,1.096386,1.011111,1.123457,1.022472,1.096386,1.400000,1.000000,1.022472,1.096386,1.034091,...,1.058140,1.123457,1.123457,1.1375,2.935484,1.213333,1.625000,1.281690,1.166667,1.491803
7,1.072289,0.988889,1.098765,1.000000,1.072289,1.369231,0.978022,1.000000,1.072289,1.011364,...,1.034884,1.098765,1.098765,1.1125,2.870968,1.186667,1.589286,1.253521,1.141026,1.459016
8,1.000000,0.922222,1.024691,0.932584,1.000000,1.276923,0.912088,0.932584,1.000000,0.943182,...,0.965116,1.024691,1.024691,1.0375,2.677419,1.106667,1.482143,1.169014,1.064103,1.360656


In [35]:
max_val = df_proportions.max().max()
min_val = df_proportions.min().min()

max_val, min_val

(2.935483870967742, 0.34065934065934067)

In [ ]:
def scale_value(x, min_val, max_val):
    return (x - min_val) / (max_val - min_val) * 9


## TEST